# Carga de audios, transcripción y almacenamiento

In [ ]:
import os
import mysql.connector
import whisper

# Paths
audio_folder = "data/audio/"
transcription_folder = "data/transcriptions/"

# Connect to MySQL
conn = mysql.connector.connect(
    host="your-mysql-server",
    user="your-username",
    password="your-password",
    database="your-database"
)
cursor = conn.cursor()

# Load Whisper model
model = whisper.load_model("medium")

for filename in os.listdir(audio_folder):
    if filename.endswith(".wav") or filename.endswith(".mp3"):
        audio_path = os.path.join(audio_folder, filename)
        transcription_path = os.path.join(transcription_folder, filename.replace(".wav", ".txt").replace(".mp3", ".txt"))

        # Transcribe audio
        result = model.transcribe(audio_path)
        text = result["text"]

        # Save transcription to a .txt file
        with open(transcription_path, "w", encoding="utf-8") as f:
            f.write(text)

        # Insert transcription into MySQL
        sql = """
        INSERT INTO transcriptions (call_id, transcription, status)
        VALUES (%s, %s, %s)
        """
        values = (filename.replace(".wav", "").replace(".mp3", ""), text, "Pending")

        cursor.execute(sql, values)
        conn.commit()

        print(f"Saved transcription: {transcription_path} & stored in database.")

cursor.close()
conn.close()


